In [1]:
import torch
from ultralytics import YOLO
import cv2 as cv
import cvzone

In [ ]:
classNames = ['Hardhat','Mask','NO-Hardhat','NO-Mask','NO-Safety west','Person','Vest','Safety','Cone','Safety Vest','machinery','vehicle']

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model = YOLO('mask_detection.pt').to(device)
video = cv.VideoCapture(0)
width = int(video.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv.CAP_PROP_FPS))

# Create a VideoWriter object to save the output video
#output_video = cv.VideoWriter("mask_detected_video.mp4", cv.VideoWriter_fourcc(*"mp4v"), fps, (width, height))
myColor = (0,0,255) 
while True:
    success , frame = video.read()
    #frame = cv.flip(frame , 1)
    if not success:
        break
    results = model(frame , stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1 , y1 , x2 , y2 = box.xyxy[0]
            x1 , y1 , x2 , y2 = int(x1) , int(y1) , int(x2) , int(y2)
            w , h = x2 - x1 , y2 - y1
            #cvzone.cornerRect(frame , (x1,y1,w,h))
            conf = round(float(box.conf[0]) , 2)
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            if currentClass == 'Mask' and conf  > 0.4:
                myColor = (0,255,0)
                cvzone.putTextRect(frame , f'{classNames[cls]} {conf}' , (max(0 , x1) , max(35,y1)) , scale=1 , thickness=1)
                cv.rectangle(frame , (x1,y1) , (x2,y2) , myColor , 3)
            if currentClass == 'NO-Mask' and conf  > 0.4:
                myColor = (0,0,255)
                cvzone.putTextRect(frame , f'{classNames[cls]} {conf}' , (max(0 , x1) , max(35,y1)) , scale=1 , thickness=1)
                cv.rectangle(frame , (x1,y1) , (x2,y2) , myColor , 3)
    cv.imshow('frame' , frame)
    #output_video.write(frame)
    if cv.waitKey(1) & 0xff==ord('q'):
        break

cv.destroyAllWindows()
video.release()
#output_video.release()